In [1]:
%load_ext sql
%sql sqlite://

'Connected: @None'

# Задание 1 

Рассмотрим 2 таблицы

A

|   name  |  key   | value
|---------|--------- | ----
| Ivanov  |   FIO  |  Иванов Иван Иванович
| Ivanov  | Phone  |  +(7) 111-1111111
| Ivanov  | email  | ivanov@ivanov.com
| Petrov  |   FIO  | Петров Петр Петрович
| Petrov  | Phone  |  +(7) 222-2222222
| Petrov  | email  | petrov@petrov.com

B

|   name  |  FIO   | Phone | email
|---------|---------|------|---
| Ivanov  | Иванов Иван Иванович | +(7) 111-1111111 | ivanov@ivanov.com
| Petrov  | Петров Петр Петрович | +(7) 222-2222222 | petrov@petrov.com


1. Как из таблицы A получить B?
2. Как из таблицы B получить A?


In [2]:
%%sql
drop table if exists tab1;
CREATE TABLE tab1 (
name varchar(100),
key varchar(100),
value varchar(100)
);

INSERT INTO tab1 VALUES('Ivanov', 'FIO', 'Иванов Иван Иванович');
INSERT INTO tab1 VALUES('Ivanov', 'Phone', '+(7) 111-1111111');
INSERT INTO tab1 VALUES('Ivanov', 'email', 'ivanov@ivanov.com');
INSERT INTO tab1 VALUES('Petrov', 'FIO', 'Петров Петр Петрович');
INSERT INTO tab1 VALUES('Petrov', 'Phone', '+(7) 222-2222222');
INSERT INTO tab1 VALUES('Petrov', 'email', 'petrov@petrov.com');


drop table if exists tab2;
CREATE TABLE tab2 (
name varchar(100),
FIO varchar(100),
Phone varchar(100),
email varchar(100)
);

INSERT INTO tab2 VALUES('Ivanov', 'Иванов Иван Иванович', '+(7) 111-1111111', 'ivanov@ivanov.com');
INSERT INTO tab2 VALUES('Petrov', 'Петров Петр Петрович', '+(7) 222-2222222', 'petrov@petrov.com');

 * sqlite://
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
Done.
Done.
1 rows affected.
1 rows affected.


[]

In [3]:
%%sql
-- tab1->tab2, 1-й вар

SELECT DISTINCT name, NTH_VALUE(value,1) OVER (PARTITION BY name) as 'FIO', 
    NTH_VALUE(value, 2) OVER (PARTITION BY name) as 'Phone', NTH_VALUE(value,3) OVER (PARTITION BY name) as 'email'
FROM
(
    SELECT *
    FROM tab1
    ORDER by key
)

 * sqlite://
Done.


name,FIO,Phone,email
Ivanov,Иванов Иван Иванович,+(7) 111-1111111,ivanov@ivanov.com
Petrov,Петров Петр Петрович,+(7) 222-2222222,petrov@petrov.com


In [4]:
%%sql
-- tab1->tab2, 2-й вар

WITH t1 as(
    SELECT *
    FROM tab1
    GROUP BY NAME
),
t2 as(
    SELECT *
    FROM tab1
    WHERE key is not "FIO"
    GROUP BY NAME
),
t3 as(
    SELECT *
    FROM tab1
    WHERE key is not "FIO" and key is not 'Phone'
    GROUP BY NAME
)

SELECT DISTINCT tab1.name, t1.value as 'FIO', t2.value as 'Phone', t3.value as "email"
FROM tab1
JOIN t1 USING(name)
JOIN t2 USING(name)
JOIN t3 USING(name)

 * sqlite://
Done.


name,FIO,Phone,email
Ivanov,Иванов Иван Иванович,+(7) 111-1111111,ivanov@ivanov.com
Petrov,Петров Петр Петрович,+(7) 222-2222222,petrov@petrov.com


In [5]:
%%sql
-- tab2->tab1, 1-й вар

SELECT name, 'FIO' as key, FIO as "value"
FROM tab2
WHERE name is "Ivanov"

UNION ALL

SELECT name, 'Phone', Phone
FROM tab2
WHERE name is "Ivanov"

UNION ALL

SELECT name, 'email', email
FROM tab2
WHERE name is "Ivanov"

UNION ALL

SELECT name, 'FIO', FIO
FROM tab2
WHERE name is "Petrov"

UNION ALL

SELECT name, 'Phone', Phone
FROM tab2
WHERE name is "Petrov"

UNION ALL

SELECT name, 'email', email
FROM tab2
WHERE name is "Petrov"

 * sqlite://
Done.


name,key,value
Ivanov,FIO,Иванов Иван Иванович
Ivanov,Phone,+(7) 111-1111111
Ivanov,email,ivanov@ivanov.com
Petrov,FIO,Петров Петр Петрович
Petrov,Phone,+(7) 222-2222222
Petrov,email,petrov@petrov.com


In [6]:
%%sql
-- tab2->tab1, 2-й вар


SELECT name, 'FIO' as key, FIO as "value"
FROM tab2

UNION

SELECT name, 'Phone', Phone
FROM tab2

UNION

SELECT name, 'email', email
FROM tab2

 * sqlite://
Done.


name,key,value
Ivanov,FIO,Иванов Иван Иванович
Ivanov,Phone,+(7) 111-1111111
Ivanov,email,ivanov@ivanov.com
Petrov,FIO,Петров Петр Петрович
Petrov,Phone,+(7) 222-2222222
Petrov,email,petrov@petrov.com


# Задание 2
Рассмотрим таблицу StudentMarks

| student_name | mark_date | mark_value
|---------|--------- | ----
| Ivanov  | 2019.01.01 | 3
| Ivanov  | 2019.01.05 | 4
| Ivanov  | 2019.02.03 | 5
| Petrov  | 2019.02.01 | 2
| Petrov  | 2019.02.16 | 4
| Petrov  | 2019.04.17 | 5

В данной таблице студенту проставляется оценка за определенную дату. Необходимо сформировать интервалы, 
за время которых оценка не менялась,
и среднюю оценку, находящуюся в данном интервале.
Текущий интервал должен заканчиваться на 5999.12.31


По исходным данным результатом таблицы будет следущее:

| client_name | mark_validfrom | mark_validto | average_mark_value
|---------|--------- | ------ | -----
| Ivanov  | 2019.01.01 | 2019.01.04 | 3
| Ivanov  | 2019.01.05 | 2019.02.02 | 3.5
| Ivanov  | 2019.02.03 | 5999.12.31 | 4
| Petrov  | 2019.02.01 | 2019.02.15 | 2
| Petrov  | 2019.02.16 | 2019.04.16 | 3
| Petrov  | 2019.04.17 | 5999.12.31 | 3.6666


In [7]:
%%sql

drop table if exists tab3;
CREATE TABLE tab3 (
student_name varchar(100),
mark_date data,
mark_value int
);

INSERT INTO tab3 VALUES('Ivanov', '2019-01-01', 3);
INSERT INTO tab3 VALUES('Ivanov', '2019-01-05', 4);
INSERT INTO tab3 VALUES('Ivanov', '2019-02-03', 5);
INSERT INTO tab3 VALUES('Petrov', '2019-02-01', 2);
INSERT INTO tab3 VALUES('Petrov', '2019-02-16', 4);
INSERT INTO tab3 VALUES('Petrov', '2019-04-17', 5);

 * sqlite://
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [8]:
%%sql

SELECT client_name, mark_validfrom, DATE(mark_validto) as mark_validto, average_mark_value
FROM
(
    SELECT student_name as client_name, mark_date as mark_validfrom,
        LEAD(DATETIME(mark_date, '-1 days'), 1, '5999-12-31') OVER(PARTITION BY student_name) as mark_validto,
        AVG(mark_value) OVER(PARTITION BY student_name ORDER BY mark_date) as average_mark_value
    FROM tab3
)

 * sqlite://
Done.


client_name,mark_validfrom,mark_validto,average_mark_value
Ivanov,2019-01-01,2019-01-04,3.0
Ivanov,2019-01-05,2019-02-02,3.5
Ivanov,2019-02-03,5999-12-31,4.0
Petrov,2019-02-01,2019-02-15,2.0
Petrov,2019-02-16,2019-04-16,3.0
Petrov,2019-04-17,5999-12-31,3.6666666666666665


# Задание 3


Рассмотрим таблицу CheatList, описывающий блокнот со шпаргалками. В данной таблице на одном листе может быть только одна шпаргалка по одному предмету. Блокнот состоит из 2N листов, пронумерованных от 0 до 2N-1. 0 лист всегда пустой, 2N-1 лист может быть как пустым, так и заполненным. В самом конце блокнота не может быть 2 подряд пустых листа. Необходимо вернуть таблицу вида (left_list, right_list), описывающий все листы в отсортированном порядке
## Пример 

| ListNumber | Subject 
| -----------|--------
|  1         | Math
|  3         | Math
|  4         | History

## Результат (должен быть без цифр 0, ..., 5)

| left_list | right_list 
| --------- | ---------
|   NULL    |  Math
|   NULL    |  Math
|   History |  NULL

In [9]:
%%sql

drop table if exists tab4;
CREATE TABLE tab4 (
ListNumber int,
Subject varchar(100)
);

INSERT INTO tab4 VALUES(1, 'Math');
INSERT INTO tab4 VALUES(3, 'Math');
INSERT INTO tab4 VALUES(4, 'History');

 * sqlite://
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [10]:
%%sql

WITH RECURSIVE tab5(n, m) AS
(
    SELECT 0 as n, "NULL" as m
    
    UNION ALL
    
    SELECT n+1, 
        CASE
        WHEN (SELECT Subject FROM tab4 WHERE ListNumber = n + 1 ) is not NULL 
            THEN (SELECT Subject FROM tab4 WHERE ListNumber = n + 1 )
        ELSE "NULL"
        END   
    FROM tab5
    WHERE   (
                CASE
                WHEN (SELECT MAX(ListNumber) FROM tab4) % 2 = 1 THEN n < (SELECT MAX(ListNumber) FROM tab4)
                ELSE n < ((SELECT MAX(ListNumber) FROM tab4) + 1)
                END
            )
),
t1 as(
    SELECT n, m
    FROM tab5
    WHERE n % 2 = 0
),
t2 as(
    SELECT n, m
    FROM tab5
    WHERE n % 2 = 1
)

SELECT t1.m as left_list, t2.m as right_list
FROM t1
JOIN t2 on t1.n + 1 = t2.n

 * sqlite://
Done.


left_list,right_list
NULL,Math
NULL,Math
History,NULL


In [11]:
%%sql

drop table if exists tab4;
CREATE TABLE tab4 (
ListNumber int,
Subject varchar(100)
);

INSERT INTO tab4 VALUES(1, 'Math');
INSERT INTO tab4 VALUES(3, 'Math');
INSERT INTO tab4 VALUES(5, 'History');

 * sqlite://
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [12]:
%%sql

WITH RECURSIVE tab5(n, m) AS
(
    SELECT 0 as n, "NULL" as m
    
    UNION ALL
    
    SELECT n+1, 
        CASE
        WHEN (SELECT Subject FROM tab4 WHERE ListNumber = n + 1 ) is not NULL 
            THEN (SELECT Subject FROM tab4 WHERE ListNumber = n + 1 )
        ELSE "NULL"
        END   
    FROM tab5
    WHERE   (
                CASE
                WHEN (SELECT MAX(ListNumber) FROM tab4) % 2 = 1 THEN n < (SELECT MAX(ListNumber) FROM tab4)
                ELSE n < ((SELECT MAX(ListNumber) FROM tab4) + 1)
                END
            )
),
t1 as(
    SELECT n, m
    FROM tab5
    WHERE n % 2 = 0
),
t2 as(
    SELECT n, m
    FROM tab5
    WHERE n % 2 = 1
)

SELECT t1.m as left_list, t2.m as right_list
FROM t1
JOIN t2 on t1.n + 1 = t2.n

 * sqlite://
Done.


left_list,right_list
NULL,Math
NULL,Math
NULL,History


# Задание 4
Используя формат xml, описанный в [dbmstools](http://dbmstools.sourceforge.net/docs/xmlschema.html), напишите функцию на языке Python, на входе который будет путь к xml, на выходе - скрипт с DDL скриптами таблиц (можно ограничиться table блоком)

In [ ]:
# XML-скрипт для проверки №1
<?xml version="1.0" encoding="UTF-8"?>
<db-schema name="Company">
        <table name="Users">
                <columns>
                        <column name="user_id"
                                type="int"
                                auto-increment="true" />
                        <column name="second_id"
                                type="int"
                                auto-increment="true" />
                        <column name="name"
                                type="VARCHAR"
                                length="50"
                                allows-null="false" />
                        <column name="email"
                                type="VARCHAR"
                                length="50"/>
                </columns>
                <constraints>
                        <primary-key name="PK_First">
                                <column-ref name="user_id"/>
                                <column-ref name="second_id"/>
                        </primary-key>
                        <unique>
                                <column-ref name="user_id"/>
                                <column-ref name="second_id"/>
                        </unique>
                </constraints>
                <indexes>
                        <index name="first_ind" unique="true">
                                <column-ref name="second_id"/>
                                <column-ref name="name"/>
                        </index>
                </indexes>
        </table>
        <table name="Flowers">
                <columns>
                        <column name="flower_id"
                                type="int"
                                auto-increment="true" />
                        <column name="name"
                                type="VARCHAR"
                                length="50"
                                allows-null="false" />
                        <column name="location"
                                type="VARCHAR"
                                length="50" />
                </columns>
                <constraints>
                        <foreign-key name="PK_First" references="Users">
                                <column-ref name="flower_id"/>
                                <referenced-column name="second_id"/>
                        </foreign-key>
                </constraints>
        </table>
</db-schema>

In [ ]:
###XML-скрипт для проверки №2
<?xml version="1.0" encoding="UTF-8"?>
<db-schema name="Football">
        <table name="Positions">
                <columns>
                        <column name="id"
                                type="int"
                                allows-null="false" />
                        <column name="name"
                                type="VARCHAR"
                                length="50"
                                allows-null="false" />
                </columns>
                <constraints>
                        <primary-key name="PK_first">
                                <column-ref name="id"/>
                        </primary-key>
                </constraints>
        </table>
        <table name="Coaches">
                <columns>
                        <column name="id"
                                type="int"
                                allows-null="false" />
                        <column name="name"
                                type="VARCHAR"
                                length="50"
                                allows-null="false" />
                </columns>
                <constraints>
                        <primary-key name="PK_first">
                                <column-ref name="id"/>
                        </primary-key>
                </constraints>
        </table>
        <table name="Teams">
                <columns>
                        <column name="id"
                                type="int"
                                allows-null="false" />
                        <column name="name"
                                type="VARCHAR"
                                length="50"
                                allows-null="false" />
                        <column name="location"
                                type="VARCHAR"
                                length="50" />
                        <column name="coach_id"
                                type="int" />
                </columns>
                <constraints>
                        <primary-key name="PK_first">
                                <column-ref name="id"/>
                        </primary-key>
                        <foreign-key name="PK_First" references="Coaches">
                                <column-ref name="coach_id"/>
                                <referenced-column name="id"/>
                        </foreign-key>
                </constraints>
        </table>
        <table name="Players">
                <columns>
                        <column name="id"
                                type="int"
                                allows-null="false" />
                        <column name="name"
                                type="VARCHAR"
                                length="50"
                                allows-null="false" />
                        <column name="age"
                                type="int" />
                        <column name="pos_id"
                                type="int" />
                        <column name="team_id"
                                type="int" />
                        <column name="start_date"
                                type="data" />
                </columns>
                <constraints>
                        <primary-key name="PK_first">
                                <column-ref name="id"/>
                        </primary-key>
                        <foreign-key name="PK_First" references="Positions">
                                <column-ref name="pos_id"/>
                                <referenced-column name="id"/>
                        </foreign-key>
                        <foreign-key name="PK_Second" references="Teams">
                                <column-ref name="team_id"/>
                                <referenced-column name="id"/>
                        </foreign-key>
                </constraints>
        </table>
        <table name="PlayersArchive">
                <columns>
                        <column name="player_id"
                                type="int" />
                        <column name="team_id"
                                type="int" />
                        <column name="start_date"
                                type="data" />
                        <column name="end_date"
                                type="data" />
                </columns>
                <constraints>
                        <foreign-key name="PK_First" references="Players">
                                <column-ref name="player_id"/>
                                <referenced-column name="id"/>
                        </foreign-key>
                        <foreign-key name="PK_Second" references="Teams">
                                <column-ref name="team_id"/>
                                <referenced-column name="id"/>
                        </foreign-key>
                </constraints>
        </table>
</db-schema>

In [13]:
import os
import xml.etree.ElementTree as ET

def tableParser(node):
    
    columns = []
    for col in node.findall('columns'):
        columns.append(col)
    
    constraints = []
    for con in node.findall('constraints'):
        constraints.append(con)
    
    indexes = []
    for i in node.findall('indexes'):
        itmp = []
        itmp2 = []
        for i1 in i:
            itmp.append(i1.get('name'))
            itmp.append(i1.get('unique'))
            for i2 in i1:
                itmp2.append(i2.get('name'))
            itmp.append(itmp2)
        indexes.append(itmp)
        
        
    cols = [] # массив для значений столбцов
    for c1 in columns:
        for c2 in c1:
            tmp = []
            tmp.append(c2.get('name'))
            tmp.append(c2.get('type'))
            tmp.append(c2.get('length'))
            tmp.append(c2.get('allows-null'))
            tmp.append(c2.get('auto-increment'))
            cols.append(tmp)
    
    primary_keys = []
    for c1 in constraints:
        for c2 in c1.findall('primary-key'):
            for c3 in c2.findall('column-ref'):
                primary_keys.append(c3.get('name'))
    
    unique = []
    for c1 in constraints:
        for c2 in c1.findall('unique'):
            for c3 in c2.findall('column-ref'):
                unique.append(c3.get('name'))

    foreign_key = []
    for c1 in constraints:
        for c2 in c1.findall('foreign-key'):
            cur = []
            cur.append(c2.get('references'))
            for c3 in c2.findall('column-ref'):
                cur.append(c3.get('name'))
            for c3 in c2.findall('referenced-column'):
                cur.append(c3.get('name'))
            foreign_key.append(cur)    
                
            
            
    return cols, primary_keys, unique, foreign_key, indexes

In [14]:
def Parser(xml_file):
    
    general_query = ""
    tree = ET.parse(xml_file)
    root = tree.getroot()
    main_name = root.get('name') # названия схемы таблиц

    tables_name = [] # названия таблиц
    tables = [] # таблицы как узлы дерева
    FLAG = 0

    for node in root.findall('table'):
        tables.append(node)
        tables_name.append(node.get('name'))

    for t in tables:

        cols = []
        primary_keys = []
        unique = []
        foreign_key = []
        indexes = []

        cols, primary_keys, unique, foreign_key, indexes = tableParser(t)

        query = "DROP TABLE if EXISTS {0};".format(t.get('name'))
        query = query + "\nCREATE TABLE {0}\n(\n".format(t.get('name'))
        u = ""
        pk = ""
        for c in cols:

            #общий вид элемента с = [name, type, length, allows-null, auto-increment]

            if c[2]:
                query = query + "{0} {1}({2})".format(c[0], c[1], c[2])
            else:
                query = query + "{0} {1}".format(c[0], c[1])

            if c[3] == "false":
                query = query + " NOT NULL"

            if c[4] == "true":
                query = query + " AUTO_INCREMENT"

            if primary_keys:
                pk = ",\nPRIMARY KEY("
                for p1 in primary_keys:
                    pk = pk + "{0}".format(p1)
                    if p1 != primary_keys[len(primary_keys) - 1]:
                        pk = pk + ", "
                pk = pk + ")"

            if unique:
                u = ",\nUNIQUE("
                for u1 in unique:
                    u = u + "{0}".format(u1)
                    if u1 != unique[len(unique) - 1]:
                        u = u + ", "
                u = u + ")"

            fk = ""
            if foreign_key:
                FLAG = 1
                for f in foreign_key:
                    fk = fk + ",\nFOREIGN KEY({0})".format(f[1]) + " REFERENCES " + "{0}({1})".format(f[0], f[2])


            if c != cols[len(cols) - 1]:
                query = query + ",\n"
            else:
                query = query + fk + pk + u + "\n);\n"

        if indexes:
            for i in indexes:
                for i1 in i[2]:
                    ind = "\nCREATE "
                    if i[1] == "true":
                        ind = ind + "UNIQUE "
                    ind = ind + "INDEX {0} ON {1}({2})".format(i1, t.get('name'), i1) + ";\n"
                    query = query + ind
        
        general_query = general_query + "\n" + query
    return("Pragma foreign_key = on;\n" + general_query)
    

In [15]:
print(Parser("TAB.xml"))

Pragma foreign_key = on;

DROP TABLE if EXISTS Users;
CREATE TABLE Users
(
user_id int AUTO_INCREMENT,
second_id int AUTO_INCREMENT,
name VARCHAR(50) NOT NULL,
email VARCHAR(50),
PRIMARY KEY(user_id, second_id),
UNIQUE(user_id, second_id)
);

CREATE UNIQUE INDEX second_id ON Users(second_id);

CREATE UNIQUE INDEX name ON Users(name);

DROP TABLE if EXISTS Flowers;
CREATE TABLE Flowers
(
flower_id int AUTO_INCREMENT,
name VARCHAR(50) NOT NULL,
location VARCHAR(50),
FOREIGN KEY(flower_id) REFERENCES Users(second_id)
);



In [16]:
%%sql

-- Проверка результата для XML-скрипта №1

Pragma foreign_key = on;

DROP TABLE if EXISTS Users;
CREATE TABLE Users
(
user_id int AUTO_INCREMENT,
second_id int AUTO_INCREMENT,
name VARCHAR(50) NOT NULL,
email VARCHAR(50),
PRIMARY KEY(user_id, second_id),
UNIQUE(user_id, second_id)
);

CREATE UNIQUE INDEX second_id ON Users(second_id);

CREATE UNIQUE INDEX name ON Users(name);

DROP TABLE if EXISTS Flowers;
CREATE TABLE Flowers
(
flower_id int AUTO_INCREMENT,
name VARCHAR(50) NOT NULL,
location VARCHAR(50),
FOREIGN KEY(flower_id) REFERENCES Users(second_id)
);


 * sqlite://
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

In [17]:
print(Parser("TAB2.xml"))

Pragma foreign_key = on;

DROP TABLE if EXISTS Positions;
CREATE TABLE Positions
(
id int NOT NULL,
name VARCHAR(50) NOT NULL,
PRIMARY KEY(id)
);

DROP TABLE if EXISTS Coaches;
CREATE TABLE Coaches
(
id int NOT NULL,
name VARCHAR(50) NOT NULL,
PRIMARY KEY(id)
);

DROP TABLE if EXISTS Teams;
CREATE TABLE Teams
(
id int NOT NULL,
name VARCHAR(50) NOT NULL,
location VARCHAR(50),
coach_id int,
FOREIGN KEY(coach_id) REFERENCES Coaches(id),
PRIMARY KEY(id)
);

DROP TABLE if EXISTS Players;
CREATE TABLE Players
(
id int NOT NULL,
name VARCHAR(50) NOT NULL,
age int,
pos_id int,
team_id int,
start_date data,
FOREIGN KEY(pos_id) REFERENCES Positions(id),
FOREIGN KEY(team_id) REFERENCES Teams(id),
PRIMARY KEY(id)
);

DROP TABLE if EXISTS PlayersArchive;
CREATE TABLE PlayersArchive
(
player_id int,
team_id int,
start_date data,
end_date data,
FOREIGN KEY(player_id) REFERENCES Players(id),
FOREIGN KEY(team_id) REFERENCES Teams(id)
);



In [18]:
%%sql

-- Проверка результата для XML-скрипта №2

Pragma foreign_key = on;

DROP TABLE if EXISTS Positions;
CREATE TABLE Positions
(
id int NOT NULL,
name VARCHAR(50) NOT NULL,
PRIMARY KEY(id)
);

DROP TABLE if EXISTS Coaches;
CREATE TABLE Coaches
(
id int NOT NULL,
name VARCHAR(50) NOT NULL,
PRIMARY KEY(id)
);

DROP TABLE if EXISTS Teams;
CREATE TABLE Teams
(
id int NOT NULL,
name VARCHAR(50) NOT NULL,
location VARCHAR(50),
coach_id int,
FOREIGN KEY(coach_id) REFERENCES Coaches(id),
PRIMARY KEY(id)
);

DROP TABLE if EXISTS Players;
CREATE TABLE Players
(
id int NOT NULL,
name VARCHAR(50) NOT NULL,
age int,
pos_id int,
team_id int,
start_date data,
FOREIGN KEY(pos_id) REFERENCES Positions(id),
FOREIGN KEY(team_id) REFERENCES Teams(id),
PRIMARY KEY(id)
);

DROP TABLE if EXISTS PlayersArchive;
CREATE TABLE PlayersArchive
(
player_id int,
team_id int,
start_date data,
end_date data,
FOREIGN KEY(player_id) REFERENCES Players(id),
FOREIGN KEY(team_id) REFERENCES Teams(id)
);

 * sqlite://
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

# Задание 5
Необходимо уметь хранить матрицы MxM внутри СУБД. Предложите способ хранения, и приведите пример перемножения двух матриц средствами SQL

In [19]:
%%sql
drop table if exists tabM1;
CREATE TABLE tabM1 (
i int,
k int,
value int
);

drop table if exists tabM2;
CREATE TABLE tabM2 (
k int,
j int,
value int
);

INSERT INTO tabM1 VALUES(0, 0, 1);
INSERT INTO tabM1 VALUES(0, 1, 2);
INSERT INTO tabM1 VALUES(0, 2, 3);
INSERT INTO tabM1 VALUES(1, 0, 4);
INSERT INTO tabM1 VALUES(1, 1, 5);
INSERT INTO tabM1 VALUES(1, 2, 6);
INSERT INTO tabM1 VALUES(2, 0, 7);
INSERT INTO tabM1 VALUES(2, 1, 8);
INSERT INTO tabM1 VALUES(2, 2, 9);

INSERT INTO tabM2 VALUES(0, 0, 1);
INSERT INTO tabM2 VALUES(0, 1, 2);
INSERT INTO tabM2 VALUES(0, 2, 3);
INSERT INTO tabM2 VALUES(1, 0, 4);
INSERT INTO tabM2 VALUES(1, 1, 5);
INSERT INTO tabM2 VALUES(1, 2, 6);
INSERT INTO tabM2 VALUES(2, 0, 7);
INSERT INTO tabM2 VALUES(2, 1, 8);
INSERT INTO tabM2 VALUES(2, 2, 9);

 * sqlite://
Done.
Done.
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

Ссылка на вольфрам с результатом перемножения:
https://www.wolframalpha.com/input/?i=%7B%7B1%2C2%2C3%7D%2C%7B4%2C5%2C6%7D%2C%7B7%2C8%2C9%7D%7D*%7B%7B1%2C2%2C3%7D%2C%7B4%2C5%2C6%7D%2C%7B7%2C8%2C9%7D%7D

In [20]:
%%sql
SELECT i, j, SUM(tabM1.value * tabM2.value) as value
FROM tabM1
JOIN tabM2 USING(k)
GROUP BY i,j;

 * sqlite://
Done.


i,j,value
0,0,30
0,1,36
0,2,42
1,0,66
1,1,81
1,2,96
2,0,102
2,1,126
2,2,150
